In [1]:
import pyaudio
import struct
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

CHUNK = 1024 * 4
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 4410

In [28]:
p = pyaudio.PyAudio()

stream = p.open(
    format = FORMAT,
    channels = CHANNELS,
    rate = RATE,
    input = True,
    output = True,
    frames_per_buffer = CHUNK
)
##data = stream.read(CHUNK)
#data
#len(data)
#data_int = struct.unpack(str(2 * CHUNK) + 'B', data) 
##data_int = np.array(struct.unpack(str(2 * CHUNK) + 'B', data), dtype = 'b')[::2] + 127
#data_int

fig,ax = plt.subplots()

x = np.arange(0,2 * CHUNK, 2)
line, = ax.plot(x, np.random.rand(CHUNK))
ax.set_ylim(0, 255)
ax.set_xlim(0, CHUNK)


while True:
    data = stream.read(CHUNK)
    data_int = np.array(struct.unpack(str(2 * CHUNK) + 'B', data), dtype = 'b')[::2] + 127
    line.set_ydata(data_int)
    fig.canvas.draw()
    fig.canvas.flush_events()

#ax.plot(data_int, '-')
##plt.show()



TclError: invalid command name "."

In [30]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
import time
from scipy.fftpack import fft
from tkinter import TclError
# use this backend to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 2                  # samples per frame
FORMAT = pyaudio.paInt16          # audio format (bytes per sample?)
CHANNELS = 1                      # single channel for microphone
RATE = 44100                      # samples per second



In [40]:
#create matplotlib figure and axes
fig, (ax,ax2) = plt.subplots(2, figsize = (15, 8))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format = FORMAT,
    channels = CHANNELS,
    rate = RATE,
    input = True,
    output = True,
    frames_per_buffer = CHUNK
)

#variable for plotting
x = np.arange(0, 2 * CHUNK, 2)
x_fft = np.linspace(0, RATE,CHUNK)

# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw = 2)
#line_fft, = ax2.plot(x_fft, np.random.rand(CHUNK), '-', lw = 2)
line_fft, = ax2.semilogx(x_fft, np.random.rand(CHUNK), '-', lw = 2)

#basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks = [0, CHUNK, 2 * CHUNK], yticks = [0, 128, 255])

ax2.set_xlim(20, RATE / 2)

#show the plot
plt.show(block = False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()


while True:
    
    # binary data
    data = stream.read(CHUNK)
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset it by 128
    data_np = np.array(data_int, dtype = 'b')[::2] + 128
    
    line.set_ydata(data_np)
    
    y_fft = fft(data_int)
    line_fft.set_ydata(np.abs(y_fft[0:CHUNK]) * 2 / (256 * CHUNK))
    
                   
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
                   
    except TclError:
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
                   
        print('stream stopped')
        print('averge frame rate = {:.0f} FPS'.format(frame_rate))
        break
    
  #  fig.canvas.draw()
  #  fig.canvas.flush_events()




stream started
stream stopped
averge frame rate = 10 FPS


In [38]:
plt.plot(np.abs(fft(np.sin(2 * x))))